In [9]:
from pycoral.pybind._pywrap_coral import GetRuntimeVersion as get_runtime_version
from pycoral.pybind._pywrap_coral import ListEdgeTpus as list_edge_tpus
print('runtime', get_runtime_version())
print('edge tpus', list_edge_tpus())

runtime BuildLabel(COMPILER=6.3.0 20170516,DATE=redacted,TIME=redacted), RuntimeVersion(14)
edge tpus [{'type': 'pci', 'path': '/dev/apex_0'}]


In [1]:
import argparse
import time

import numpy as np
from PIL import Image
from pycoral.adapters import classify
from pycoral.adapters import common
from pycoral.utils.dataset import read_label_file
from pycoral.utils.edgetpu import make_interpreter

In [2]:
labels = read_label_file('../pycoral/test_data/inat_bird_labels.txt')
model_file_name = '../pycoral/test_data/mobilenet_v2_1.0_224_inat_bird_quant_edgetpu.tflite'
top_k = 1
threshold = 0.0

In [12]:
interpreter = make_interpreter(*model_file_name.split('@'))
interpreter.allocate_tensors()
# Model must be uint8 quantized
if common.input_details(interpreter, 'dtype') != np.uint8:
    raise ValueError('Only support uint8 input type.')

size = common.input_size(interpreter)
image = Image.open('../pycoral/test_data/parrot.jpg').convert('RGB').resize(size, Image.ANTIALIAS)
print('size', size)

size (224, 224)


In [4]:
# efficiency; we use 1e-5 below instead of absolute zero).
params = common.input_details(interpreter, 'quantization_parameters')
scale = params['scales']
zero_point = params['zero_points']
mean = 128.0
std = 128.0
if abs(scale * std - 1) < 1e-5 and abs(mean - zero_point) < 1e-5:
  # Input data does not require preprocessing.
  common.set_input(interpreter, image)
else:
  # Input data requires preprocessing
  normalized_input = (np.asarray(image) - mean) / (std * scale) + zero_point
  np.clip(normalized_input, 0, 255, out=normalized_input)
  common.set_input(interpreter, normalized_input.astype(np.uint8))

# Run inference
print('----INFERENCE TIME----')
print('Note: The first inference on Edge TPU is slow because it includes',
      'loading the model into Edge TPU memory.')
for _ in range(5):
  start = time.perf_counter()
  interpreter.invoke()
  inference_time = time.perf_counter() - start
  classes = classify.get_classes(interpreter, top_k, threshold)
  print('%.1fms' % (inference_time * 1000))

print('-------RESULTS--------')
for c in classes:
  print('%s: %.5f' % (labels.get(c.id, c.id), c.score))



----INFERENCE TIME----
Note: The first inference on Edge TPU is slow because it includes loading the model into Edge TPU memory.
15.3ms
4.1ms
3.5ms
3.4ms
3.4ms
-------RESULTS--------
Ara macao (Scarlet Macaw): 0.75781
